> need to run ch1 to load data.

We first distinguish between Discard and ICT contracts. Therefore we first get rid of some Discard to reduce computing load.

In [1]:
import os
import sqlite3
import numpy as np
import pandas as pd
import sklearn

In [2]:
import nltk
import re

In [3]:
#### ICT Contracts in the last 3 years
contract = pd.read_csv('3ycontract.csv') # main contract tabel
(contract.head())

,Cnid,idSupplier,Title,PDate,austender_cat,austender_supplier,internalcomment,EntityName,IdCat
0,CN1148392,9446,Provision of Web Hosting & Bandwidth Costs,26/05/2016 3:03:00 PM,Components for information technology or broad...,Eye Media Australia Pty Ltd,NaN,EYE MEDIA AUSTRALIA PTY LTD,14
1,CN2707381,44012,Extension of the iVEC UQ RDSI Existing Disk Ar...,12/08/2015 12:00:00 AM,Components for information technology or broad...,DataDirect Networks,Recovered from Discards; Supplier provides sto...,DDN AUSTRALIA PTY LIMITED,9
2,CN2745091,9793,IT Professional Services,5/01/2015 10:01:00 AM,Information technology consultation services,Redcore Pty Ltd,NaN,REDCORE PTY LTD,11
3,CN2748341,6061,ICT Contractors,14/01/2015 11:28:00 AM,Computer services,Southern Cross Computing Pty Ltd,NaN,SOUTHERN CROSS COMPUTING PTY LIMITED,13
4,CN2748891,21412,ICT Contractors,5/01/2015 11:21:00 AM,Computer services,Horizon Consulting Pty Ltd,NaN,HORIZON CONSULTING PTY LTD,13


In [4]:
#### discards in the last 3 years.
discards = pd.read_csv('3ydiscards.csv')
(discards.head())

,Cnid,Publish Date,Description,Category,SON ID,Supplier Name,Comment
0,CN1094702,26/05/2016 3:10:00 PM,Provision of Financial Consultancy Services,Accounting services,SON241274,Synergy Group Australia Ltd,NaN
1,CN1907701,1/02/2016 11:07:00 AM,Internal Audit,Audit services,NaN,Moore Stephens,NaN
2,CN2032461,26/05/2016 2:25:00 PM,Registar Liaison Officer,Temporary personnel services,NaN,Attorney General's Department CPM,NaN
3,CN2439052,12/02/2015 10:19:00 AM,Printing and Distribution 2014,Printing,NaN,CanPrint Communications Pty Ltd,NaN
4,CN2542831,25/03/2015 11:38:00 AM,Management Services,Management support services,SON1180562,DFP Recruitment Services Pty Ltd,NaN


# Stage 2: Modeling fitting description based model.
We assume that Master contract match and ICT supplier categorisation is reliable.
We start from modelling descriptions.
#### Extract useful columns

In [5]:
# contract
contract_des = contract[['Cnid','Title','IdCat']]

In [6]:
contract_des.columns=['Cnid','Description','idCat']
contract_des.head()

,Cnid,Description,idCat
0,CN1148392,Provision of Web Hosting & Bandwidth Costs,14
1,CN2707381,Extension of the iVEC UQ RDSI Existing Disk Ar...,9
2,CN2745091,IT Professional Services,11
3,CN2748341,ICT Contractors,13
4,CN2748891,ICT Contractors,13


#### Handle empty contract field to avoid error in later steps.

In [7]:
contract_des[contract_des.Description.isnull()]
contract_des[~(contract_des.Description).isnull()].head()

,Cnid,Description,idCat
0,CN1148392,Provision of Web Hosting & Bandwidth Costs,14
1,CN2707381,Extension of the iVEC UQ RDSI Existing Disk Ar...,9
2,CN2745091,IT Professional Services,11
3,CN2748341,ICT Contractors,13
4,CN2748891,ICT Contractors,13


In [8]:
contract_des = contract_des[~(contract_des.Description).isnull()] 

In [9]:
# contract_des.fillna(value="NODesc",inplace=True) # alternatively fillna.

In [10]:
len(contract_des) # 55701 raw ICT contracts in total.

55701

In [11]:
discards_des =discards[['Cnid','Description']]
discards_des[discards_des.Description.isnull()]

,Cnid,Description
70659,CN3328465,NaN
91960,CN3360215,NaN


In [12]:
discards_des.fillna(value="NODesc",inplace=True) # remember to fillna otherwise it would cause errors in the code below.
discards_des.head()
len(discards_des)  # 3 times the amount of IT contract.
# categories can fillna after wards when joined with contracts when needed.

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


155371

Give discards category of 99

In [13]:
idDis = pd.Series(np.ones(len(discards_des)))
discards_des = pd.concat([discards_des,idDis],axis=1,ignore_index=True)

In [14]:
discards_des.columns= ['Cnid','Description','idCat']
discards_des.idCat = 99
discards_des.head()

,Cnid,Description,idCat
0,CN1094702,Provision of Financial Consultancy Services,99
1,CN1907701,Internal Audit,99
2,CN2032461,Registar Liaison Officer,99
3,CN2439052,Printing and Distribution 2014,99
4,CN2542831,Management Services,99


In [15]:
# Balance train test set by downsampling discards.
from sklearn.utils import resample

In [16]:
discards_des2 = resample(discards_des, 
                                 replace=False,    # sample without replacement
                                 n_samples=55701,     # to match minority class
                                 random_state=123) # reproducible results

In [17]:
len(discards_des)

155371

In [18]:
len(discards_des2)

55701

### make train test set for each.
Separate df into 20% and rest. So is discards

In [19]:
# need to import sub packages explicitly
from sklearn import model_selection

In [20]:
contract_train,contract_test = sklearn.model_selection.train_test_split(contract_des,test_size=0.2, random_state=1)

In [21]:
discards_train,discards_test = sklearn.model_selection.train_test_split(discards_des2,test_size=0.2, random_state=1)

__When description only method, join two dataframes into one for both contract and discards. __

In [22]:
df_train = pd.concat([contract_train,discards_train],join='outer',ignore_index=True)
df_train.head()

,Cnid,Description,idCat
0,CN3339767,Development of Data Management System,17
1,CN3419927,Software Upgrade,17
2,CN3378926,Specialist Technical Advisor,11
3,CN3432035,DVA ICTSB 2016/157 - System Analyst - ICT Plan...,13
4,CN3295947,ICT Services,17


In [23]:
df_train["idCat"].value_counts()

99    44560
13    14280
17     4636
18     3266
7      2385
11     2328
14     1883
28     1778
15     1440
6      1034
31      909
1       849
19      848
12      734
34      713
52      673
5       628
8       548
40      501
21      489
4       449
32      443
29      427
9       418
33      389
43      388
30      349
27      303
37      289
42      285
22      242
20      238
25      187
44      144
36       87
Name: idCat, dtype: int64

In [24]:
len(df_train)/2*1.25

55700.0

In [25]:
df_train.to_csv("df_train.csv")

In [26]:
df_test = pd.concat([contract_test,discards_test],join='outer',ignore_index=True)
df_test.head()
df_test.index = df_test['Cnid']
df_test.drop(['Cnid'],axis=1,inplace=True)

In [27]:
df_test.head()

,Description,idCat
Cnid,,
CN3348181,Program Management Services,17
CN3110212,Distributed Computing Central Services,14
CN3442793,Automated Test Engineer,13
CN3422590,Software Services,28
CN3428960,VC System,43


### Change to two categories: Insert one more column,if category is not 99 or 52, IT. 
Note: The reason categorise yetIT here instead of before spliting into training test set is due to consideration of other multi class scenarios.

In [28]:
df_train.insert(loc=2,column='yesIT',value= 1)

In [29]:
len(df_train.columns)

4

In [30]:
df_test.insert(loc=2,column='yesIT',value= 1)

In [31]:
df_train.head()

,Cnid,Description,yesIT,idCat
0,CN3339767,Development of Data Management System,1,17
1,CN3419927,Software Upgrade,1,17
2,CN3378926,Specialist Technical Advisor,1,11
3,CN3432035,DVA ICTSB 2016/157 - System Analyst - ICT Plan...,1,13
4,CN3295947,ICT Services,1,17


In [32]:
df_test.head()

,Description,idCat,yesIT
Cnid,,,
CN3348181,Program Management Services,17,1
CN3110212,Distributed Computing Central Services,14,1
CN3442793,Automated Test Engineer,13,1
CN3422590,Software Services,28,1
CN3428960,VC System,43,1


In [33]:
df_train.yesIT[df_train.idCat.isin([52,99])]=0

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
df_test.yesIT[df_test.idCat.isin([52,99])]=0

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
print('NON-IT',
    df_train[df_train.yesIT==0].head(),'\n','\n',
     len(df_train[df_train.yesIT==0]),'\n')
print('IT',df_train[df_train.yesIT==1].head(),'\n','\n',
 len(df_train[df_train.yesIT==1]))

NON-IT           Cnid                                        Description  yesIT  \
289  CN2964562             Project Manager - Questacon Facilities      0   
344  CN3281327                               Information Services      0   
396  CN3240442  Development of Optimised Usage & Upkeep Cycle ...      0   
515  CN3283166  Graphic design services to update marketing an...      0   
648  CN3281409                               Labour Hire Services      0   

     idCat  
289     52  
344     52  
396     52  
515     52  
648     52   
 
 45233 

IT         Cnid                                        Description  yesIT  idCat
0  CN3339767              Development of Data Management System      1     17
1  CN3419927                                   Software Upgrade      1     17
2  CN3378926                       Specialist Technical Advisor      1     11
3  CN3432035  DVA ICTSB 2016/157 - System Analyst - ICT Plan...      1     13
4  CN3295947                                       IC

In [37]:
df_test.to_csv("df_test.csv")

### Step2: Normalisation of words - for now just focus on training set. Also apply same transition on test set.

__ We need to standardise words with the following features: __

* retain upper case features for certain words with different meanings when lowercased like ['IT',] - replace with word: infotech as there is no other matches.

Tokenize each description before lemmatizing them.

In [37]:
df_train.head(10)

,Cnid,Description,yesIT,idCat
0,CN3339767,Development of Data Management System,1,17
1,CN3419927,Software Upgrade,1,17
2,CN3378926,Specialist Technical Advisor,1,11
3,CN3432035,DVA ICTSB 2016/157 - System Analyst - ICT Plan...,1,13
4,CN3295947,ICT Services,1,17
5,CN3396801,Information retrieval systems,1,29
6,CN2979762,Printers & Licences,1,8
7,CN3421256,Laptops - high end,1,4
8,CN3330935,Provision of Electronic Hardware and Component...,1,14
9,CN3318967-A1,Provision of IT Contractor Services,1,13


In [38]:
# replace 'IT' with 'infotech'
IT_regex = r'(?<=(\b))IT(?=(\b))'
df_train.Description = df_train.Description.str.replace(IT_regex, 'infotech',case=True)

In [39]:
# make sure no IT is missed.
print(df_train[df_train.Description.str.contains('infotech')].count())
print(df_train[df_train.Description.str.contains('IT')].count())
print(df_train[df_train.Description.str.contains(' IT ')].head())

Cnid           3420
Description    3420
yesIT          3420
idCat          3420
dtype: int64
Cnid           453
Description    453
yesIT          453
idCat          453
dtype: int64
Empty DataFrame
Columns: [Cnid, Description, yesIT, idCat]
Index: []


In [40]:
# change the index to CNID to ensure easy tracking.
df_train.head()

,Cnid,Description,yesIT,idCat
0,CN3339767,Development of Data Management System,1,17
1,CN3419927,Software Upgrade,1,17
2,CN3378926,Specialist Technical Advisor,1,11
3,CN3432035,DVA ICTSB 2016/157 - System Analyst - ICT Plan...,1,13
4,CN3295947,ICT Services,1,17


In [41]:
df_train.index = df_train.Cnid
df_train.drop(['Cnid'],axis =1,inplace=True)
df_train.head()

,Description,yesIT,idCat
Cnid,,,
CN3339767,Development of Data Management System,1,17
CN3419927,Software Upgrade,1,17
CN3378926,Specialist Technical Advisor,1,11
CN3432035,DVA ICTSB 2016/157 - System Analyst - ICT Plan...,1,13
CN3295947,ICT Services,1,17


In [42]:
df_train.to_csv('df_train.csv')

### We extract description column to process before joining back to the dataframe.

Lemmatization considers upper case as special terminologies, so still lower cased every thing.

In [43]:
# before tokenize, replace all non-alpha numeric characters with a space.
regex2 = r'[^a-zA-Z]'
df_train.Description = df_train.Description.str.replace(regex2,' ',case=True)

# tokenize sentences.
title_tokenised = df_train.Description.str.lower().apply(nltk.word_tokenize)
title_tokenised.head()

Cnid
CN3339767          [development, of, data, management, system]
CN3419927                                  [software, upgrade]
CN3378926                     [specialist, technical, advisor]
CN3432035    [dva, ictsb, system, analyst, ict, planning, a...
CN3295947                                      [ict, services]
Name: Description, dtype: object

In [44]:
# Create function to Lemmatize each entry but do not return to sentence.
def lem_cont(tokens):
    wnl = nltk.WordNetLemmatizer()
    lemmed = [wnl.lemmatize(t) for t in tokens]
    return lemmed

In [45]:
title_token_lem = title_tokenised.apply(lem_cont)

In [46]:
title_token_lem.head()

Cnid
CN3339767          [development, of, data, management, system]
CN3419927                                  [software, upgrade]
CN3378926                     [specialist, technical, advisor]
CN3432035    [dva, ictsb, system, analyst, ict, planning, a...
CN3295947                                       [ict, service]
Name: Description, dtype: object

In [47]:
# export token lem
title_token_lem.to_csv('title_token_lem.csv',header='x_train_token_lem')

> Try Stemming

> following code is raw text so that they are not executed

## Step3: Removal of stop words, non-aphabetic characters

> start by loading the 'title_lemtoken.csv' that contains the tokenised entries.

Remove: stop words, numbers, punctuations.

In [48]:
from nltk.corpus import stopwords

In [49]:
def removestopword(text):
    stopwremoved = [w for w in text if not w in stopwords.words('english')]
    return stopwremoved

In [50]:
title_token_lem_nostopword = title_token_lem.apply(removestopword)

In [51]:
title_token_lem_nostopword.head()

Cnid
CN3339767              [development, data, management, system]
CN3419927                                  [software, upgrade]
CN3378926                     [specialist, technical, advisor]
CN3432035    [dva, ictsb, system, analyst, ict, planning, b...
CN3295947                                       [ict, service]
Name: Description, dtype: object

## Step4: join the tokens back together for storing as well as sklearn process.

In [52]:
# save to disk title_lem_nostop as csv - so next time can start from here.
#title_lem_nostop.to_excel(excel_writer='title_lem_nostop.xlsx',header='x_train_token_lem')
title_token_lem_nostopword.to_csv('title_token_lem_nostopword.csv',header='x_train_token_lem')
y_trainval = df_train.yesIT
y_trainval.to_csv('y_trainval.csv',header = 'yesIT')

In [57]:
type(y_trainval)

pandas.core.series.Series

In [58]:
y_trainval.value_counts()

0    45233
1    43887
Name: yesIT, dtype: int64

## Produce result csv for next stage processing. - This result requires further lemmatization and TFIDF transformation.

In [53]:
x_trainval = title_token_lem_nostopword.apply(lambda w: ' '.join(w)) # join rows together to make a long string.

In [54]:
x_trainval.head()

Cnid
CN3339767                   development data management system
CN3419927                                     software upgrade
CN3378926                         specialist technical advisor
CN3432035    dva ictsb system analyst ict planning business...
CN3295947                                          ict service
Name: Description, dtype: object

In [55]:
x_trainval.to_csv('x_trainval.csv',header='lemnostopwDesc')